# Классификация текстов с использованием Наивного Байесовского Классификатора

## Задание 1 (1 балл)

Откройте данные. Узнайте, сколько в них спам- и не спам-писем. Визуализируйте полученные соотношение подходящим образом.

In [2]:
import pandas as pd
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import ComplementNB
from sklearn import metrics


In [3]:
email_data = pd.read_csv('data/spam_or_not_spam.csv')
email_data.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [4]:
describer = {
  0: 'не спам',
  1: 'спам'
}
ratio_info = email_data.label.value_counts().to_frame().reset_index()
ratio_info.label = ratio_info.label.map(describer)
ratio_info.set_index('label', inplace=True)
ratio_info


,count
label,
не спам,2500
спам,500


In [5]:
fig = px.pie(ratio_info, 
             values = 'count',
             names = ratio_info.index,
             hole=.3,
             )
fig.update_layout(
            title_text="Соотношение писем, относительно спама"
    )
fig.show()

## Задание 2 (2 балла)

Вам необходимо предобработать ваши данные и перевести их в векторный вид. Подгрузим необходимый модуль:

Замените в данных все пустые строки и строки, состоящие из пробелов, на пропуски (NaN). После этого удалите из данных все строки, в которых наблюдаются пропущенные значения.

In [18]:
# удалим записи, где в тексте пистма содержаться только пробелы, 
# а затем записи содержащие nan эначения
spam_data = email_data[~email_data.email.str.strip().eq('')].dropna()

Переводим данные в векторный вид:

In [7]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(spam_data["email"])

Определите, сколько теперь признаков в нашем наборе данных:

In [8]:
print(f'В наборе данных {X.shape[1]} признаков.')

В наборе данных 34116 признаков.


## Задание 3 (2 балла)

Определите целевую переменную и признаки:

In [9]:
y = spam_data['label']

Разделите выборку на обучающую и тестовую, используя стратифицированное разбиение (параметр `stratify` установите в значение вектора ответов y) размер тестовой выборки (`test_size`) возьмите как 0.25, параметр `random_state` определите со значением 42:

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

Рассчитайте среднее значение целевой переменной по тестовой выборке:

In [11]:
y_test.mean().round(3)

0.165

## Задание 4 (3 балла)

Определите и обучите подходящий алгоритм с параметром alpha = 0.01

In [12]:
model = ComplementNB(alpha=0.01)
model.fit(X_train, y_train)

ComplementNB(alpha=0.01)

Оцените результат с точки зрения всех известных вам метрик (не менее трёх):

In [13]:
y_pred = model.predict(X_test)

print('Accuracy: {:.3f}'.format(metrics.accuracy_score(y_test, y_pred)))
print('Precision: {:.3f}'.format(metrics.precision_score(y_test, y_pred)))
print('Recall: {:.3f}'.format(metrics.recall_score(y_test, y_pred)))
print('F1 score: {:.3f}'.format(metrics.f1_score(y_test, y_pred)))

Accuracy: 0.988
Precision: 1.000
Recall: 0.927
F1 score: 0.962


Нарисуйте ROC-кривую:

In [17]:
# получим предсказание вероятности правильного определения спам/не спам
# и на этих жанных построим ROC кривую
y_pred_proba = model.predict_proba(X_test)[:,1]
false_rate, true_rate, _ = metrics.roc_curve(y_test, y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)


fig = px.line(x=false_rate, 
              y=true_rate,
              markers=True
              )
fig.update_layout(title='ROC кривая для текущей модели',
                   xaxis_title='False positive rate',
                   yaxis_title='True positive rate')
fig.add_annotation(
        x=.9,
        y=.05,
        text=f"значение auc: {round(auc,6)}",
        font=dict(
            size=12,
            color="#ffffff"
            ),
        align="center",
        ax=-2,
        borderpad=4,
        bgcolor="#166071",
        opacity=0.7,
        showarrow=False
        )

fig.show()

## Задание 5 (3 балла)

Переберите несколько значений alpha с помощью кросс-валидации. Оцените, зависит ли от этого параметра качество классификации.

In [15]:
# создадим диапазон для параметра alpha, 
# с последующим использованием в подборе гиперпараметров модели
range_ = list(map(lambda x : round(x/100000,4), list(range(0, 100010, 100))[::-1]))
param_grid = {'alpha': range_}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs = -1
)

grid_search.fit(X_train, y_train)
y_test_pred = grid_search.predict(X_test)
print("Наилучшие значения гиперпараметров: {}".format(max_alpha := grid_search.best_params_))

Наилучшие значения гиперпараметров: {'alpha': 0.435}


In [16]:
# создадим dataframe на основе результатов подбора параметров,
# это упростит нам отрисовку графика
grid_search_results = pd.DataFrame(grid_search.cv_results_)
fig = px.line(
              grid_search_results,
              x='param_alpha', 
              y='mean_test_score',
            )
fig.update_layout(title='Изменение качества предсказания модели от alpha',
                   xaxis_title='Параметр alpha',
                   yaxis_title='Средний счет на тестовых данных',
                   )
fig.update_yaxes(range = [0.97,.995])
fig.add_annotation(
        x=.435,
        y=.992,
        text=f"макс. счет при alpha:  {max_alpha['alpha']}<br>{grid_search.best_score_:43.6f}",
        font=dict(
            size=12,
            color="#ffffff"
            ),
        align="center",
        ax=20,
        borderpad=4,
        bgcolor="#166071",
        opacity=0.7,
        showarrow=True,
        arrowhead=5,
        arrowsize=1,
        arrowwidth=1,
        arrowcolor="#000000",
        ay=-30,        
        )

fig.show()

Как мы можем наблюдать, мы можем добиваться, как улучшения предсказания нашей модели, так и ухудшения в зависимости от параметра alpha.

Наилучший результат при alpha = 0.435